In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as A
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,Dense
from img_helper import channel_extractor_c1

A.set_image_data_format('channels_last')
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

dataset_npy_path = "Dataset npys/"
trained_models_path = "trained models/"

In [2]:
# Image parameters
image_width = 100
image_height = image_width
no_of_channels = 3

**Loading dataset**

In [3]:
training_HSV_Low = np.load(os.path.join(dataset_npy_path,"training_HSV_Low.npy"))
validation_HSV_Low = np.load(os.path.join(dataset_npy_path,"validation_HSV_Low.npy"))
training_HSV_High = np.load(os.path.join(dataset_npy_path,"training_HSV_High.npy"))
validation_HSV_High = np.load(os.path.join(dataset_npy_path,"validation_HSV_High.npy"))
training_HSV_High_noisy = np.load(os.path.join(dataset_npy_path,"training_HSV_High_noisy.npy"))
validation_HSV_High_noisy = np.load(os.path.join(dataset_npy_path,"validation_HSV_High_noisy.npy"))
print("data loaded")

data loaded


**Additional functions**

In [4]:
import matplotlib.pyplot as plt
def plot_image(image):
    plt.imshow(image, cmap="gray")
    plt.axis("off")#Displays the original images and their reconstructions
def show_reconstructions(model, images=validation_HSV_Low, n_images=5):
    reconstructions = model.predict(images[:n_images,:,:,2])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
      plt.subplot(2, n_images, 1 + image_index)
      plot_image(images[image_index,:,:,2])
      plt.subplot(2, n_images, 1 + n_images + image_index)
      plot_image(reconstructions[image_index])

**Stacked auto encoder model**

In [5]:
# Parameters of stacked encoder
SAE_Epochs = 1000
SAE_Batch_Size = 512
input_images_V = Input(shape = (image_width, image_height))
def stacked_auto_encoder_model(input_images_V):

    D_3 = tf.keras.layers.Dense(image_width, activation='relu')(input_images_V)

    return D_3
stacked_ae = keras.models.Model(input_images_V,stacked_auto_encoder_model(input_images_V))
stacked_ae.compile(loss="mean_squared_error",optimizer='Adam')
stacked_ae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100)          0         
_________________________________________________________________
dense (Dense)                (None, 100, 100)          10100     
Total params: 10,100
Trainable params: 10,100
Non-trainable params: 0
_________________________________________________________________


**Fitting the model by training it**

In [6]:
stacked_ae_h = stacked_ae.fit(training_HSV_Low[:,:,:,2], training_HSV_High[:,:,:,2], batch_size = 1250,
                              epochs=800, verbose=1, validation_data=(validation_HSV_Low[:,:,:,2],
                                                                     validation_HSV_High[:,:,:,2]))

Train on 5831 samples, validate on 1458 samples
Epoch 1/800
5831/5831 [==============================] - 7s 1ms/step - loss: 0.1608 - val_loss: 0.1416
Epoch 2/800
5831/5831 [==============================] - 2s 393us/step - loss: 0.1333 - val_loss: 0.1153
Epoch 3/800
5831/5831 [==============================] - 2s 384us/step - loss: 0.1087 - val_loss: 0.0939
Epoch 4/800
5831/5831 [==============================] - 2s 402us/step - loss: 0.0902 - val_loss: 0.0804
Epoch 5/800
5831/5831 [==============================] - 4s 673us/step - loss: 0.0792 - val_loss: 0.0731
Epoch 6/800
5831/5831 [==============================] - 2s 403us/step - loss: 0.0727 - val_loss: 0.0674
Epoch 7/800
5831/5831 [==============================] - 2s 390us/step - loss: 0.0670 - val_loss: 0.0621
Epoch 8/800
5831/5831 [==============================] - 2s 377us/step - loss: 0.0620 - val_loss: 0.0580
Epoch 9/800
5831/5831 [==============================] - 2s 356us/step - loss: 0.0584 - val_loss: 0.0551
Epoch 10/

Epoch 78/800
5831/5831 [==============================] - 3s 429us/step - loss: 0.0187 - val_loss: 0.0180
Epoch 79/800
5831/5831 [==============================] - 3s 432us/step - loss: 0.0185 - val_loss: 0.0179
Epoch 80/800
5831/5831 [==============================] - 2s 417us/step - loss: 0.0185 - val_loss: 0.0178
Epoch 81/800
5831/5831 [==============================] - 2s 426us/step - loss: 0.0184 - val_loss: 0.0178
Epoch 82/800
5831/5831 [==============================] - 2s 424us/step - loss: 0.0183 - val_loss: 0.0177
Epoch 83/800
5831/5831 [==============================] - 3s 456us/step - loss: 0.0182 - val_loss: 0.0176
Epoch 84/800
5831/5831 [==============================] - 3s 454us/step - loss: 0.0181 - val_loss: 0.0175
Epoch 85/800
5831/5831 [==============================] - 3s 447us/step - loss: 0.0180 - val_loss: 0.0174
Epoch 86/800
5831/5831 [==============================] - 3s 479us/step - loss: 0.0179 - val_loss: 0.0173
Epoch 87/800
5831/5831 [======================

Epoch 155/800
5831/5831 [==============================] - 3s 441us/step - loss: 0.0146 - val_loss: 0.0143
Epoch 156/800
5831/5831 [==============================] - 2s 344us/step - loss: 0.0145 - val_loss: 0.0143
Epoch 157/800
5831/5831 [==============================] - 2s 427us/step - loss: 0.0145 - val_loss: 0.0142
Epoch 158/800
5831/5831 [==============================] - 2s 422us/step - loss: 0.0145 - val_loss: 0.0142
Epoch 159/800
5831/5831 [==============================] - 3s 440us/step - loss: 0.0145 - val_loss: 0.0142
Epoch 160/800
5831/5831 [==============================] - 3s 519us/step - loss: 0.0144 - val_loss: 0.0142
Epoch 161/800
5831/5831 [==============================] - 4s 693us/step - loss: 0.0144 - val_loss: 0.0141
Epoch 162/800
5831/5831 [==============================] - 3s 446us/step - loss: 0.0144 - val_loss: 0.0141
Epoch 163/800
5831/5831 [==============================] - 3s 477us/step - loss: 0.0143 - val_loss: 0.0141
Epoch 164/800
5831/5831 [============

5831/5831 [==============================] - 2s 417us/step - loss: 0.0130 - val_loss: 0.0129
Epoch 232/800
5831/5831 [==============================] - 2s 416us/step - loss: 0.0130 - val_loss: 0.0128
Epoch 233/800
5831/5831 [==============================] - 2s 414us/step - loss: 0.0130 - val_loss: 0.0128
Epoch 234/800
5831/5831 [==============================] - 2s 414us/step - loss: 0.0130 - val_loss: 0.0128
Epoch 235/800
5831/5831 [==============================] - 2s 418us/step - loss: 0.0129 - val_loss: 0.0128
Epoch 236/800
5831/5831 [==============================] - 2s 369us/step - loss: 0.0129 - val_loss: 0.0128
Epoch 237/800
5831/5831 [==============================] - 2s 412us/step - loss: 0.0129 - val_loss: 0.0128
Epoch 238/800
5831/5831 [==============================] - 2s 420us/step - loss: 0.0129 - val_loss: 0.0128
Epoch 239/800
5831/5831 [==============================] - 2s 419us/step - loss: 0.0129 - val_loss: 0.0128
Epoch 240/800
5831/5831 [==========================

5831/5831 [==============================] - 2s 413us/step - loss: 0.0122 - val_loss: 0.0122
Epoch 308/800
5831/5831 [==============================] - 2s 412us/step - loss: 0.0122 - val_loss: 0.0122
Epoch 309/800
5831/5831 [==============================] - 2s 415us/step - loss: 0.0122 - val_loss: 0.0122
Epoch 310/800
5831/5831 [==============================] - 2s 417us/step - loss: 0.0122 - val_loss: 0.0122
Epoch 311/800
5831/5831 [==============================] - 2s 417us/step - loss: 0.0122 - val_loss: 0.0121
Epoch 312/800
5831/5831 [==============================] - 2s 419us/step - loss: 0.0122 - val_loss: 0.0121
Epoch 313/800
5831/5831 [==============================] - 2s 414us/step - loss: 0.0122 - val_loss: 0.0121
Epoch 314/800
5831/5831 [==============================] - 2s 412us/step - loss: 0.0122 - val_loss: 0.0121
Epoch 315/800
5831/5831 [==============================] - 2s 416us/step - loss: 0.0122 - val_loss: 0.0121
Epoch 316/800
5831/5831 [==========================

...

5831/5831 [==============================] - 2s 425us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 460/800
5831/5831 [==============================] - 2s 414us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 461/800
5831/5831 [==============================] - 2s 413us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 462/800
5831/5831 [==============================] - 3s 594us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 463/800
5831/5831 [==============================] - 3s 504us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 464/800
5831/5831 [==============================] - 2s 418us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 465/800
5831/5831 [==============================] - 2s 415us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 466/800
5831/5831 [==============================] - 2s 417us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 467/800
5831/5831 [==============================] - 2s 420us/step - loss: 0.0117 - val_loss: 0.0117
Epoch 468/800
5831/5831 [==========================

5831/5831 [==============================] - 2s 409us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 536/800
5831/5831 [==============================] - 2s 410us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 537/800
5831/5831 [==============================] - 2s 423us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 538/800
5831/5831 [==============================] - 2s 409us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 539/800
5831/5831 [==============================] - 2s 407us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 540/800
5831/5831 [==============================] - 2s 351us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 541/800
5831/5831 [==============================] - 2s 390us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 542/800
5831/5831 [==============================] - 2s 411us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 543/800
5831/5831 [==============================] - 2s 412us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 544/800
5831/5831 [==========================

5831/5831 [==============================] - 3s 430us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 612/800
5831/5831 [==============================] - 2s 413us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 613/800
5831/5831 [==============================] - 2s 411us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 614/800
5831/5831 [==============================] - 2s 418us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 615/800
5831/5831 [==============================] - 2s 409us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 616/800
5831/5831 [==============================] - 2s 340us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 617/800
5831/5831 [==============================] - 2s 400us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 618/800
5831/5831 [==============================] - 2s 413us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 619/800
5831/5831 [==============================] - 2s 415us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 620/800
5831/5831 [==========================

5831/5831 [==============================] - 2s 411us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 688/800
5831/5831 [==============================] - 2s 412us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 689/800
5831/5831 [==============================] - 2s 407us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 690/800
5831/5831 [==============================] - 2s 420us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 691/800
5831/5831 [==============================] - 2s 407us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 692/800
5831/5831 [==============================] - 2s 413us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 693/800
5831/5831 [==============================] - 2s 417us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 694/800
5831/5831 [==============================] - 2s 411us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 695/800
5831/5831 [==============================] - 2s 412us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 696/800
5831/5831 [==========================

5831/5831 [==============================] - 3s 467us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 764/800
5831/5831 [==============================] - 3s 478us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 765/800
5831/5831 [==============================] - 3s 459us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 766/800
5831/5831 [==============================] - 3s 461us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 767/800
5831/5831 [==============================] - 3s 466us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 768/800
5831/5831 [==============================] - 3s 475us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 769/800
5831/5831 [==============================] - 3s 467us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 770/800
5831/5831 [==============================] - 3s 458us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 771/800
5831/5831 [==============================] - 2s 413us/step - loss: 0.0116 - val_loss: 0.0116
Epoch 772/800
5831/5831 [==========================

**Saving the model**

In [10]:
import pickle
stacked_ae.save(os.path.join(trained_models_path,"stack_encoder_100.h5"))
print("Saved stack_encoder model to disk")

with open('trainHistoryDict_stack.pkl', 'wb') as file_pi:
        pickle.dump(stacked_ae_h.history, file_pi)

Saved stack_encoder model to disk


**Convolutional auto encoder**

In [11]:

# Convolutional encoder parameters
CAE_epochs = 20
batch_size_CAE = 128
CAE_inp_image = Input(shape = (image_width, image_height, 1))

# Convolutional encoder architecture
def conv_auto_encoder_model(CAE_inp_image):
	c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(CAE_inp_image)
	p1 = MaxPooling2D(pool_size=(2, 2))(c1)
	c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
	up1 = UpSampling2D((2,2))(c2)
	c3 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up1)
	return c3

# Build convolutional model
c_e = Model(CAE_inp_image, conv_auto_encoder_model(CAE_inp_image))
c_e.compile(loss='mean_squared_error', optimizer='Adam')



**Fitting the model by training**

In [12]:
# Train convolutional model
c_e_h = c_e.fit(channel_extractor_c1(training_HSV_High_noisy, image_width, 2), channel_extractor_c1(training_HSV_High, image_width, 2),
					batch_size=batch_size_CAE,
					epochs=CAE_epochs,
					verbose=1,
					validation_data=(channel_extractor_c1(validation_HSV_High_noisy, image_width, 2), channel_extractor_c1(validation_HSV_High, image_width, 2)))

Train on 5831 samples, validate on 1458 samples
Epoch 1/20
5831/5831 [==============================] - 644s 110ms/step - loss: 0.0236 - val_loss: 0.0087
Epoch 2/20
5831/5831 [==============================] - 707s 121ms/step - loss: 0.0082 - val_loss: 0.0070
Epoch 3/20
5831/5831 [==============================] - 727s 125ms/step - loss: 0.0072 - val_loss: 0.0066
Epoch 4/20
5831/5831 [==============================] - 692s 119ms/step - loss: 0.0069 - val_loss: 0.0064
Epoch 5/20
5831/5831 [==============================] - 564s 97ms/step - loss: 0.0067 - val_loss: 0.0062
Epoch 6/20
5831/5831 [==============================] - 566s 97ms/step - loss: 0.0066 - val_loss: 0.0063
Epoch 7/20
5831/5831 [==============================] - 573s 98ms/step - loss: 0.0064 - val_loss: 0.0060
Epoch 8/20
5831/5831 [==============================] - 580s 100ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 9/20
5831/5831 [==============================] - 547s 94ms/step - loss: 0.0062 - val_loss: 0.0059
Ep

**Saving convolutional auto encoder part**

In [13]:
c_e.save(os.path.join(trained_models_path,"c_e.h5"))
print("Saved convolutional auto encoder model to disk")

with open('trainHistoryDict_conv.pkl', 'wb') as file_pi:
        pickle.dump(c_e_h.history, file_pi)

Saved convolutional auto encoder model to disk
